In [4]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

from time import sleep

In [9]:
class Request:
    selenium_reties = 0
    
    def __init(self, url):
        self.url = url
        
    def get_selenium_res(self, class_name):
        try:
            software_names = [SoftwareName.CHROME.value]
            operating_systems = [OperatingSystem.WINDOWS.value,
                                OperatingSystem.LINUX.value]
            
            user_agent_rotator = UserAgent(software_names=software_names,
                                           operating_systems=operating_systems,
                                           limit=100)
            
            user_agent = user_agent_rotator.get_random_user_agent()
            
            chrome_options = Options()
            chrome_options.add_argument('--headless') #When running a lot of request, bc it doesn't really open a browser it takes of
            # some load of the pc BUT going headless may help the site flag you as scraper, if get banned try without
            chrome_options.add_argument('--no-sandbox') #Only way to run headless
            chrome_options.add_argument('--window-size=1920,1080')
            chrome_options.add_argument('--disable-gpu') #Normally only for windows
            chrome_options.add_argument(f'user-agent={user_agent}')
            
            #Proxy settings
            PROXY = 'https://gate.smartproxy.com:7000'
            
            if not IS_GAEPRODUCTION_ENV:
                chrome_options.binary_location = '/Users/corentin/Documents/chromedriver'
                
            prox = Proxy()
            prox.proxy_type = ProxyType.MANUAL
            prox.autodetect = False
            capabilities = webdriver.DesiredCapabilities.CHROME
            prox.http_proxy = PROXY
            prox.ssl_proxy = PROXY
            prox.add_to_capabilities(capabilities)
            
            # browser = webdriver.Chrome(chrome_options=chrome_options, 
            #                            desired_capabilities=capabilities)
            browser = webdriver.Chrome(chrome_options=chrome_options)
            
            # when testing proxies
            # browser.get('http://lumtest.com/myip.json')
            browser.get(self.url)
            
            time_to_wait = 90
            try:
                WebDriverWait(browser, time_to_wait).until(
                    EC.presence_of_element_located((By.CLASS_NAME, class_name)))
            finally:
                browser.maximize_window()
                page_html = browser.page_source
                browser.close()
                return page_html
        except (TimeoutException, WebDriverException):
            self.logger.error(traceback.format_exc())
            sleep(6)
            self.selenium_retries += 1
            self.logger.info('Selenium retry #: ' + str(self.selenium_retries))
            return self.get_selenium_res(class_name)
            